In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi']=150

# Load scripts from parent path
import sys, os
sys.path.insert(0, os.path.abspath('..'))

## Load Data

In [2]:
# from scripts.processing_copy_Maryam import load_train_data, process_data, add_store_info, add_week_month_info, add_begginin
import scripts.processing as scr
train_raw = scr.load_train_data()
train = scr.add_week_month_info(train_raw)
train = scr.add_beginning_end_month(train)
train = scr.process_data(train)
train = scr.add_store_info(train)
# cols_to_drop = ['Open', 'StateHoliday', 'Assortment']
# train = train.drop(cols_to_drop, axis=1)

#train.head()
#X_train, X_test, y_train, y_test = scr.prepare_data(train, test_size = 0.2)

## Prepare train/test data

In [3]:
X = train.copy(deep=True).drop(columns=["Sales"])
y = train.loc[:, "Sales"]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [14]:
X_train.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,week,month,end_of_month,beginning_of_month,StoreType,Assortment,CompetitionDistance,Store_Sales_mean,Store_Customers_mean
243225,57.0,2.0,0.0,d,0.0,46,11,0.022453,0.141113,d,c,420.0,9710.620690,880.862069
90112,1067.0,6.0,0.0,d,0.0,17,4,0.575451,0.000277,d,c,12020.0,4312.436364,374.018182
270028,108.0,1.0,1.0,d,0.0,51,12,0.070963,0.054817,d,c,2190.0,9614.428571,746.481481
232756,211.0,2.0,0.0,d,0.0,44,10,0.765853,0.000017,a,c,350.0,9708.642857,1129.821429
289960,692.0,1.0,0.0,d,0.0,3,1,0.030926,0.113669,a,a,40.0,5270.303571,665.089286


## Encoding features

In [5]:
import category_encoders as ce

#ord_enode = ce.OrdinalEncoder(cols=['Foreign_schooled',
 #                                    'Past_Disciplinary_Action','Previous_IntraDepartmental_Movement'])

#base_encode = ce.BaseNEncoder(cols=['State_Of_Origin','Marital_Status'])
# one_hot = ce.OneHotEncoder(cols=['StateHoliday'])
#one_hot = ce.OneHotEncoder(cols=['StoreType', 'Assortment','StateHoliday'])
# one_hot = ce.OneHotEncoder(cols=['StoreType'])
target_encode  = ce.TargetEncoder(cols = ['Store','StoreType', 'Assortment','StateHoliday','week','month'])
#ordinal_encode = ce.OrdinalEncoder(cols=['StateHoliday'])

In [7]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()

In [12]:
regLGBMGridSearch.estimator.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'target_encode', 'model', 'target_encode__cols', 'target_encode__drop_invariant', 'target_encode__handle_missing', 'target_encode__handle_unknown', 'target_encode__min_samples_leaf', 'target_encode__return_df', 'target_encode__smoothing', 'target_encode__verbose', 'model__boosting_type', 'model__class_weight', 'model__colsample_bytree', 'model__importance_type', 'model__learning_rate', 'model__max_depth', 'model__min_child_samples', 'model__min_child_weight', 'model__min_split_gain', 'model__n_estimators', 'model__n_jobs', 'model__num_leaves', 'model__objective', 'model__random_state', 'model__reg_alpha', 'model__reg_lambda', 'model__silent', 'model__subsample', 'model__subsample_for_bin', 'model__subsample_freq'])

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
pipe = Pipeline(steps=[ 
                ('target_encode', target_encode),
               ('model',model)])                           
                                
parameters = {'model__n_estimators': [400,500,600], 'model__max_depth': [16,20], 'model__num_leaves': [35,40,45]}
regLGBMGridSearch = GridSearchCV(pipe, parameters, cv=5, n_jobs=6, verbose=1)
grid_search = regLGBMGridSearch.fit(X_train, y_train) 

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done  90 out of  90 | elapsed:  5.0min finished
/home/maryam/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [14]:
print("Best parameter (CV score=%0.3f):" % grid_search.best_score_)

Best parameter (CV score=0.930):


In [16]:
grid_search.best_estimator_

Pipeline(steps=[('target_encode',
                 TargetEncoder(cols=['Store', 'StoreType', 'Assortment',
                                     'StateHoliday', 'week', 'month'])),
                ('model',
                 LGBMRegressor(max_depth=20, n_estimators=600, num_leaves=45))])

In [17]:
from scripts.processing import metric
y_pred = grid_search.best_estimator_.predict(X_test)
error_regLGBM = metric(y_test.values, y_pred)

In [18]:
error_regLGBM

12.185305642500737

/home/maryam/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/maryam/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/maryam/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/maryam/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/maryam/anaconda3/lib/p